# Automated ML

Import Dependencies.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.data.dataset_factory import TabularDatasetFactory

import pandas as pd
from sklearn.model_selection import train_test_split

from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
import joblib

## Dataset

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'heart_disease_automl'

experiment=Experiment(ws, experiment_name)
run = experiment.start_logging()

In [3]:
cpu_cluster_name = "capstone-comp"
# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Jobrunning......
Running


In [4]:
URL = "https://raw.githubusercontent.com/guptaa98/Microsoft_Azure_Nanodegree_Projects/main/Capstone%20Project/heart.csv"
ds = TabularDatasetFactory.from_delimited_files(path=URL)

In [5]:
x = ds.to_pandas_dataframe().dropna()
y = x.pop('target')

x_train, x_test, y_train, y_test = train_test_split(x, y)
dataset = pd.concat([x_train,y_train],axis=1)

In [45]:
dataset.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
264,54,1,0,110,206,0,0,108,1,0.00,1,1,2,0
229,64,1,2,125,309,0,1,131,1,1.80,1,0,3,0
111,57,1,2,150,126,1,1,173,0,0.20,2,1,3,1
8,52,1,2,172,199,1,1,162,0,0.50,2,0,3,1
290,61,1,0,148,203,0,1,161,0,0.00,2,1,3,0


## AutoML Configuration

In [71]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 20,
                   "primary_metric": "accuracy"}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task="classification",
                             training_data=dataset,
                             label_column_name='target',
                             n_cross_validations=5,
                            **automl_settings)

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

In [8]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

In [9]:
import joblib
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()
print(best_run)
print(best_run_metrics['accuracy'])
print(fitted_model)

Run(Experiment: heart_disease_automl,
Id: AutoML_cb6204eb-8a5f-403b-8f29-2d7c7e1f9fac_41,
Type: None,
Status: Completed)
0.8766183574879227
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                objective=None,
                                                                                                random_state=None,
                         

In [68]:
best_run_metrics

{'precision_score_micro': 0.8766183574879227,
 'precision_score_macro': 0.8783393098129941,
 'recall_score_macro': 0.8769184132958495,
 'AUC_micro': 0.9366434222502276,
 'accuracy': 0.8766183574879227,
 'f1_score_macro': 0.874175838430682,
 'norm_macro_recall': 0.7538368265916991,
 'recall_score_micro': 0.8766183574879227,
 'AUC_macro': 0.9369528597339691,
 'f1_score_micro': 0.8766183574879227,
 'AUC_weighted': 0.9369528597339691,
 'average_precision_score_weighted': 0.9417381907428929,
 'recall_score_weighted': 0.8766183574879227,
 'weighted_accuracy': 0.8765753185456356,
 'f1_score_weighted': 0.8766616000968005,
 'average_precision_score_micro': 0.9403524390695699,
 'log_loss': 0.37379690086193545,
 'average_precision_score_macro': 0.9395381541061237,
 'matthews_correlation': 0.755046432971812,
 'balanced_accuracy': 0.8769184132958495,
 'precision_score_weighted': 0.8832309350112553,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_cb6204eb-8a5f-403b-8f29-2d7c7e1f9fac

In [10]:
#TODO: Save the best model
joblib.dump(fitted_model, "best_automl_model.pkl")

['best_automl_model.pkl']

## Model Deployment

Register the model, Create an inference config and deploy the model as a web service.

In [11]:
from azureml.core.model import Model
model = Model.register(model_path="best_automl_model.pkl",
                       model_name="best_automl_model",
                       workspace = ws)

Registering model best_automl_model


In [12]:
best_run.get_file_names()

['accuracy_table',
 'confusion_matrix',
 'explanation/9a2340be/classes.interpret.json',
 'explanation/9a2340be/expected_values.interpret.json',
 'explanation/9a2340be/features.interpret.json',
 'explanation/9a2340be/global_names/0.interpret.json',
 'explanation/9a2340be/global_rank/0.interpret.json',
 'explanation/9a2340be/global_values/0.interpret.json',
 'explanation/9a2340be/local_importance_values.interpret.json',
 'explanation/9a2340be/per_class_names/0.interpret.json',
 'explanation/9a2340be/per_class_rank/0.interpret.json',
 'explanation/9a2340be/per_class_values/0.interpret.json',
 'explanation/9a2340be/rich_metadata.interpret.json',
 'explanation/9a2340be/visualization_dict.interpret.json',
 'explanation/f1f3b58d/classes.interpret.json',
 'explanation/f1f3b58d/eval_data_viz.interpret.json',
 'explanation/f1f3b58d/expected_values.interpret.json',
 'explanation/f1f3b58d/features.interpret.json',
 'explanation/f1f3b58d/global_names/0.interpret.json',
 'explanation/f1f3b58d/global_rank/0.interpret.json',
 'explanation/f1f3b58d/global_values/0.interpret.json',
 'explanation/f1f3b58d/local_importance_values.interpret.json',
 'explanation/f1f3b58d/per_class_names/0.interpret.json',
 'explanation/f1f3b58d/per_class_rank/0.interpret.json',
 'explanation/f1f3b58d/per_class_values/0.interpret.json',
 'explanation/f1f3b58d/rich_metadata.interpret.json',
 'explanation/f1f3b58d/visualization_dict.interpret.json',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

In [14]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [15]:
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'myenv.yml')

In [26]:
from azureml.core import Environment
env = Environment.from_conda_specification(name = 'myenv.yml', file_path="myenv.yml")

In [33]:
#from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

TODO: In the cell below, deploy the model as a web service and send a request to the web service you deployed to test it.

In [41]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                       memory_gb = 1,
                                                       auth_enabled=True, 
                                                       enable_app_insights=True)

service = Model.deploy(ws, "best-model", [model], inference_config, deployment_config)
service.wait_for_deployment(True)

print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [43]:
print(service.scoring_uri)

http://39be0d2e-b043-430b-9817-53c31d5f93a6.southcentralus.azurecontainer.io/score


In [44]:
print(service.get_logs)

<bound method Webservice.get_logs of AciWebservice(workspace=Workspace.create(name='quick-starts-ws-135497', subscription_id='5a4ab2ba-6c51-4805-8155-58759ad589d8', resource_group='aml-quickstarts-135497'), name=best-model, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://39be0d2e-b043-430b-9817-53c31d5f93a6.southcentralus.azurecontainer.io/score, properties={}, created_by={'hasInferenceSchema': 'True', 'hasHttps': 'False'})>


In [46]:
dataset.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
264,54,1,0,110,206,0,0,108,1,0.00,1,1,2,0
229,64,1,2,125,309,0,1,131,1,1.80,1,0,3,0
111,57,1,2,150,126,1,1,173,0,0.20,2,1,3,1
8,52,1,2,172,199,1,1,162,0,0.50,2,0,3,1
290,61,1,0,148,203,0,1,161,0,0.00,2,1,3,0


In [50]:
import requests
import json
# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = 'lXjzyhSKevCvgqNyx3zLzSgqIAeOhBax'

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
            "age": 54,
            "sex" : 1,
            "cp" : 0,
            "trestbps" : 110,
            "chol" : 206,
            "fbs" : 0,
            "restecg":0,
            "thalach":108,
            "exang":1,
            "oldpeak":0.00,
            "slope":1,
            "ca":1,
            "thal":2
          },
          {
            
            "age": 6,
            "sex" : 1,
            "cp" : 2,
            "trestbps" : 125,
            "chol" : 309,
            "fbs" : 0,
            "restecg":1,
            "thalach":131,
            "exang":1,
            "oldpeak":1.80,
            "slope":1,
            "ca":0,
            "thal":3          
          }
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

{"result": [0, 0]}


TODO: In the cell below, print the logs of the web service and delete the service

In [51]:
print(service.get_logs())

2021-01-20T19:13:28,231766900+00:00 - gunicorn/run 
2021-01-20T19:13:28,239846800+00:00 - rsyslog/run 
2021-01-20T19:13:28,239974400+00:00 - iot-server/run 
2021-01-20T19:13:28,251248100+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_42712a5a2a2843357b09d2908e48f300/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_42712a5a2a2843357b09d2908e48f300/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_42712a5a2a2843357b09d2908e48f300/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_42712a5a2a2843357b09d2908e48f300/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_42712a5a2a2843357b09d2908e48f300/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [52]:
service.delete()